In [1]:
with open("../input.txt") as f:
    input = [line.strip() for line in f.readlines()]
# print(input)

with open("../sample-input.txt") as f:
    sample_input = [line.strip() for line in f.readlines()]
print(f"Input: {sample_input}")

Input: ['..F7.', '.FJ|.', 'SJ.L7', '|F--J', 'LJ...']


In [2]:
import numpy as np

# tile mask
tile_mask = {
    '|' : 0b001,
    '-' : 0b010,
    'L' : 0b011,
    'J' : 0b100,
    '7' : 0b101,
    'F' : 0b110,
    'S' : 0b111,
    '.' : 0b000,
}
def tile_from_mask(mask):
    for k, v in tile_mask.items():
        if v == mask:
            return k
    return '?'

neighbor_masks = {
    tile_mask['|'] : np.array([[0, 1, 0], [0, 0, 0], [0, 1, 0]]), # up and down
    tile_mask['-'] : np.array([[0, 0, 0], [1, 0, 1], [0, 0, 0]]), # left and right
    tile_mask['L'] : np.array([[0, 1, 0], [0, 0, 1], [0, 0, 0]]), # right and up
    tile_mask['J'] : np.array([[0, 1, 0], [1, 0, 0], [0, 0, 0]]), # left and up
    tile_mask['7'] : np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0]]), # left and down
    tile_mask['F'] : np.array([[0, 0, 0], [0, 0, 1], [0, 1, 0]]), # right and down
    tile_mask['S'] : np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]]), # all
}

In [3]:
src = input.copy()

def get_valid_neighbors(row, col, grid):
    # Get the 3x3 neighborhood
    neighborhood = grid[row-1:row+2, col-1:col+2]
    mask = neighbor_masks[grid[row, col]]
    return np.argwhere((neighborhood * mask) > 0) - 1 + [row, col]

def update_distances(grid, dist_grid, curr_dist):
    rows, cols = np.where(dist_grid == curr_dist)
    for row, col in zip(rows, cols):
        neighbors = get_valid_neighbors(row, col, grid)
        for n_row, n_col in neighbors:
            if np.isnan(dist_grid[n_row, n_col]):
                dist_grid[n_row, n_col] = curr_dist + 1

grid = np.array([list(row) for row in input])
mask_grid = np.vectorize(tile_mask.get)(grid)
padded_mask_grid = np.pad(mask_grid, pad_width=1, mode='constant', constant_values=0)

# Initialize distance grid
dist_grid = np.full_like(padded_mask_grid, np.nan, dtype=float)
start_pos = np.argwhere(padded_mask_grid == tile_mask['S'])[0]
curr_dist = 0
dist_grid[tuple(start_pos)] = curr_dist

# check cardinals from start_pos for matching neighbors THAT RECIPROCATE.
# Specification guarantees we only have to care about this once.
neighbors = get_valid_neighbors(*start_pos, padded_mask_grid)
curr_dist += 1
for n_row, n_col in neighbors:
    if np.all(np.any(get_valid_neighbors(n_row, n_col, padded_mask_grid) == start_pos, axis=0)):
        dist_grid[n_row, n_col] = curr_dist

while np.any(np.isnan(dist_grid)) and np.any(dist_grid == curr_dist):
    update_distances(padded_mask_grid, dist_grid, curr_dist)
    curr_dist += 1

# print(dist_grid)
print(f"Max distance in pipe from start: {int(np.nanmax(dist_grid))}")

Max distance in pipe from start: 6867
